In [1]:
import ccxt
import pandas as pd
import numpy as np

In [6]:
from utils import fetchOHLCV

df = fetchOHLCV('BTC/USDT', '1h', '2024-06-01', 500)

df

,Timestamp,Open,High,Low,Close,Volume
Datetime,,,,,,
2024-06-01 00:00:00+08:00,1717171200000,67215.58,67352.41,66670.00,67200.01,2882.03187
2024-06-01 01:00:00+08:00,1717174800000,67200.01,67459.66,67160.30,67342.77,885.94352
2024-06-01 02:00:00+08:00,1717178400000,67342.77,67444.47,67170.00,67414.88,952.07983
2024-06-01 03:00:00+08:00,1717182000000,67414.88,67999.00,67342.24,67681.00,1643.46289
2024-06-01 04:00:00+08:00,1717185600000,67681.00,67798.00,67600.00,67709.04,526.31866
...,...,...,...,...,...,...
2024-06-21 15:00:00+08:00,1718953200000,64627.64,64739.09,64190.76,64201.63,1033.34054
2024-06-21 16:00:00+08:00,1718956800000,64201.63,64372.15,63522.75,63639.64,2349.97676
2024-06-21 17:00:00+08:00,1718960400000,63639.64,64300.00,63564.37,64100.00,1803.08001


In [3]:
o = df['Open'].to_numpy()
h = df['High'].to_numpy()
l = df['Low'].to_numpy()
c = df['Close'].to_numpy()
v = df['Volume'].to_numpy()

In [4]:
from VolumeWeightedAveragePrice import VolumeWeightedAveragePrice

Bt1 = VolumeWeightedAveragePrice('anchored_daily', 8, 24)
Bt2 = VolumeWeightedAveragePrice('anchored_weekly', 8, 24 * 7)

for i in range(len(h)):
    Bt1.update(i, df.index, o, h, l, c, v)
    Bt2.update(i, df.index, o, h, l, c, v)

pd.DataFrame(Bt2.vwap)

,value,conf_i,conf_timestamp
0,NaN,0,2024-06-01 00:00:00+08:00
1,NaN,1,2024-06-01 01:00:00+08:00
2,NaN,2,2024-06-01 02:00:00+08:00
3,NaN,3,2024-06-01 03:00:00+08:00
4,NaN,4,2024-06-01 04:00:00+08:00
...,...,...,...
95,68721.934957,95,2024-06-04 23:00:00+08:00
96,68817.197288,96,2024-06-05 00:00:00+08:00
97,68851.199632,97,2024-06-05 01:00:00+08:00
98,68916.106821,98,2024-06-05 02:00:00+08:00


In [5]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

daily_vwap_df = pd.DataFrame(Bt1.vwap)
weekly_vwap_df = pd.DataFrame(Bt2.vwap)

# Create figure
fig = make_subplots(rows=2, cols=1, vertical_spacing=0.35, row_heights=[0.7, 0.3])

# Add candlestick chart
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df["Open"],
    high=df["High"],
    low=df["Low"],
    close=df["Close"],
    name="Candlestick"
), row=1, col=1)

# # Add buy signals (green arrows)
# fig.add_trace(go.Scatter(
#     x=records_df["entry_timestamp"],
#     y=records_df["entry_price"] * 0.9,
#     mode="markers",
#     marker=dict(symbol="triangle-up", color="green", size=10),
#     name="Buy Signal"
# ), row=1, col=1)

# # Add sell signals (red arrows)
# fig.add_trace(go.Scatter(
#     x=records_df["exit_timestamp"],
#     y=records_df["exit_price"] * 1.1,
#     mode="markers",
#     marker=dict(symbol="triangle-down", color="red", size=10),
#     name="Sell Signal"
# ), row=1, col=1)

fig.add_trace(go.Scatter(
    x=daily_vwap_df['conf_timestamp'],
    y=daily_vwap_df['value'],
    mode='lines',
    name='VWAP',
    line=dict(color='yellow')
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=weekly_vwap_df['conf_timestamp'],
    y=weekly_vwap_df['value'],
    mode='lines',
    name='VWAP',
    line=dict(color='blue')
), row=1, col=1)

# # Add cumulative returns line chart
# fig.add_trace(go.Scatter(
#     x=df.index,
#     y=(df['Close']-df.iloc[0]['Close'])/df.iloc[0]['Close']*100,
#     mode='lines',
#     name='Equity Curve (%)',
#     line=dict(color='blue')
# ), row=2, col=1)

# # Add cumulative returns line chart
# fig.add_trace(go.Scatter(
#     x=df.index,
#     y=equity_curve,
#     mode='lines',
#     name='Equity Curve (%)',
#     line=dict(color='orange')
# ), row=2, col=1)

# Update layout
fig.update_layout(
    title="Candlestick Chart with Buy and Sell Signals",
    xaxis_title="Datetime",
    yaxis_title="Price",
    xaxis_rangeslider_visible=True,
    xaxis2_title="Datetime",
    yaxis2_title="Equity Curve (%)",
    xaxis2_rangeslider_visible=True,
    height=800
)

# Show figure
fig.show()